## [Bayesian Factorization Machines with Stan and R](https://nyhackr.blob.core.windows.net/presentations/A_Common_Model_Separated_by_Two_Disciplines-Adam_Lauretig.pdf)

### [Mastering Shiny](https://mastering-shiny.org/basic-intro.html)

In [1]:
#shiny::runApp('./shine')

In [4]:
library(MASS)
library(Matrix)
library(trialr)

In [6]:
library(rstan)
options(mc.cores = parallel::detectCores())
rstan_options(auto_write = TRUE)

Loading required package: StanHeaders

Loading required package: ggplot2

rstan (Version 2.21.7, GitRev: 2e1f913d3ca3)

For execution on a local, multicore CPU with excess RAM we recommend calling
options(mc.cores = parallel::detectCores()).
To avoid recompilation of unchanged Stan programs, we recommend calling
rstan_options(auto_write = TRUE)



### Generating Simulation Data

In [7]:
set.seed(111)

In [8]:
N <- 100
group_1 <- paste0('i', 1:N)

In [9]:
group_1

[1] "i1"   "i2"   "i3"   "i4"   "i5"   "i6"   "i7"   "i8"   "i9"   "i10" 
 [11] "i11"  "i12"  "i13"  "i14"  "i15"  "i16"  "i17"  "i18"  "i19"  "i20" 
 [21] "i21"  "i22"  "i23"  "i24"  "i25"  "i26"  "i27"  "i28"  "i29"  "i30" 
 [31] "i31"  "i32"  "i33"  "i34"  "i35"  "i36"  "i37"  "i38"  "i39"  "i40" 
 [41] "i41"  "i42"  "i43"  "i44"  "i45"  "i46"  "i47"  "i48"  "i49"  "i50" 
 [51] "i51"  "i52"  "i53"  "i54"  "i55"  "i56"  "i57"  "i58"  "i59"  "i60" 
 [61] "i61"  "i62"  "i63"  "i64"  "i65"  "i66"  "i67"  "i68"  "i69"  "i70" 
 [71] "i71"  "i72"  "i73"  "i74"  "i75"  "i76"  "i77"  "i78"  "i79"  "i80" 
 [81] "i81"  "i82"  "i83"  "i84"  "i85"  "i86"  "i87"  "i88"  "i89"  "i90" 
 [91] "i91"  "i92"  "i93"  "i94"  "i95"  "i96"  "i97"  "i98"  "i99"  "i100"

In [10]:
J <- 20
group_2 <- paste0('j', 1:J)

In [11]:
group_2

[1] "j1"  "j2"  "j3"  "j4"  "j5"  "j6"  "j7"  "j8"  "j9"  "j10" "j11" "j12"
[13] "j13" "j14" "j15" "j16" "j17" "j18" "j19" "j20"

In [12]:
K <- 5

In [13]:
predictors <- expand.grid(group_1=group_1, group_2=group_2)

In [14]:
predictors

group_1,group_2
<fct>,<fct>
i1,j1
i2,j1
i3,j1
i4,j1
i5,j1
i6,j1
i7,j1
i8,j1
i9,j1


In [15]:
X_mat <- sparse.model.matrix(~ factor(group_1) + factor(group_2) -1, data=predictors)

In [16]:
predictors_as_numeric <- cbind(as.numeric(factor(predictors[,1])), as.numeric(factor(predictors[,2])))

In [17]:
betas <- matrix(rnorm(n=ncol(X_mat), 0,2))

In [18]:
linear_predictors = X_mat %*% betas  # y

### [LKJ](https://yingqijing.medium.com/lkj-correlation-distribution-in-stan-29927b69e9be)

#### if eta > 1, the correlation values in correlation matrices are going to centered around 0. higher eta indicate no correlations (converge to identity correlation matrix).

![](https://miro.medium.com/max/1400/1*v2LSgSMjYwCjWYpXtk437g.webp)

In [19]:
gamma_omega <- rlkjcorr(n = 1, K = K, eta = 0.9)#gamma_omega_prior)

In [20]:
delta_omega <- rlkjcorr(n=1, K=K, eta=0.5)

In [21]:
gammas <- mvrnorm(n=N, mu=rep(0,K), Sigma = gamma_omega)

In [22]:
deltas <- mvrnorm(n=J, mu=rep(0,K), Sigma = delta_omega)

In [23]:
factor_terms = matrix(NA, nrow=nrow(linear_predictors), ncol=1)

In [24]:
dim(factor_terms)

[1] 2000    1

In [25]:
for (i in 1:nrow(predictors)){
    g1 <- as.character(predictors[i,1])
    g1 <- as.numeric(substring(g1, 2, nchar(g1)))

    g2 <- as.character(predictors[i,2])
    g2 <- as.numeric(substring(g2, 2, nchar(g2)))

    #print(paste0(g1, g2))
    factor_terms[i, ] =  matrix(gammas[g1,], nrow=1) %*% matrix(deltas[g2,], ncol=1)   
}

In [26]:
y <- linear_predictors + factor_terms + rnorm(n=nrow(linear_predictors), 0, 0.1)

In [35]:
data_list = list(
    N = N,
    J = J,
    K = K,
    X = predictors_as_numeric,
    y = y[,1],
    beta_sigma = 0.05,
    y_sigma = 0.02
)

In [29]:
model = stan_model('../Stan/Models/factorization.stan')

recompiling to avoid crashing R session



In [39]:
remove(fit)
fit <- sampling(object = model,
                data = data_list,
                init = "random",
                control = list(adapt_delta = 0.95),
                chains = 4,
                iter = 1000,
                warmup = 500,
                thin = 1,
                verbose = TRUE)